In [1]:
pip install requests beautifulsoup4


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# necessary imports
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
# Define the URL of the e-commerce site (Books to Scrape)
url = "http://books.toscrape.com/"

# Send an HTTP request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Print the title of the page to verify
    print(soup.title.string)
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")



    All products | Books to Scrape - Sandbox



In [4]:
# Extract product data
products = soup.find_all("article", class_="product_pod")

# Lists to store extracted data
book_titles = []
book_prices = []
book_availability = []

# Loop through each product and extract the relevant information
for product in products:
    title = product.h3.a["title"]
    price = product.find("p", class_="price_color").text.strip()
    availability = product.find("p", class_="instock availability").text.strip()
    
    # Append data to lists
    book_titles.append(title)
    book_prices.append(price)
    book_availability.append(availability)

# Create a DataFrame to store the extracted data
data = {
    "Title": book_titles,
    "Price": book_prices,
    "Availability": book_availability
}
df = pd.DataFrame(data)

# Display the DataFrame
df.head()


,Title,Price,Availability
0,A Light in the Attic,£51.77,In stock
1,Tipping the Velvet,£53.74,In stock
2,Soumission,£50.10,In stock
3,Sharp Objects,£47.82,In stock
4,Sapiens: A Brief History of Humankind,£54.23,In stock


In [5]:
# Define the output file name
output_file = "scraped_books.csv"

# Save the DataFrame to a CSV file
df.to_csv(output_file, index=False)

print(f"Scraped data saved to {output_file}")


Scraped data saved to scraped_books.csv


In [2]:
# Necessary Imports
from flask import Flask, request, jsonify
from pinecone import Pinecone
from PIL import Image
import pytesseract
import io

# Initialize Pinecone
pc = Pinecone(api_key="2d7cf166-3b12-4801-8728-f46884148e74")
INDEX_NAME = "product-recommendations"
index = pc.Index(INDEX_NAME)

print("Pinecone serverless vector database setup complete.")

# Initialize Flask App and Define Recommendation Function
app = Flask(__name__)

def get_recommendations(query):
    results = index.query(queries=[query], top_k=5)
    recommended_products = [res['id'] for res in results['matches'][0]]
    return recommended_products

# Endpoint 1 for Text-Based Queries
@app.route('/recommend', methods=['POST'])
def recommend():
    query = request.json.get('query')
    if not query:
        return jsonify({'error': 'Query is required'}), 400
    recommendations = get_recommendations(query)
    return jsonify({'recommendations': recommendations})

# Endpoint 2 for OCR-Based Queries
@app.route('/ocr-recommend', methods=['POST'])
def ocr_recommend():
    if 'image' not in request.files:
        return jsonify({'error': 'No image provided'}), 400
    image_file = request.files['image']
    image = Image.open(io.BytesIO(image_file.read()))
    gray_image = image.convert('L')
    extracted_text = pytesseract.image_to_string(gray_image)
    if extracted_text.strip() == "":
        return jsonify({'error': 'No meaningful text extracted from the image'}), 400
    recommendations = get_recommendations(extracted_text)
    return jsonify({
        'extracted_text': extracted_text,
        'recommendations': recommendations
    })

# Cell 6: Run the Flask App
if __name__ == '__main__':
    app.run(port=5001)


Pinecone serverless vector database setup complete.
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [05/Jul/2024 07:48:24] "GET / HTTP/1.1" 404 -
